## Apply all Cell-Health Models to Training and Testing Sets

**Gregory Way, 2019**

In [1]:
import os
import pandas as pd
from joblib import load

from scripts.ml_utils import load_train_test, load_models

In [2]:
%matplotlib inline

In [3]:
def apply_model(model, feature, train_x, test_x):
    """
    Apply model to training and testing matrix
    """
    pred_train_df = (
        pd.DataFrame(model.predict(train_x), columns=["score"])
        .assign(profiles=train_x.index,
                Metadata_data_type="train",
                model=feature)
    )
    pred_test_df = (
        pd.DataFrame(model.predict(test_x), columns=["score"])
        .assign(profiles=test_x.index,
                Metadata_data_type="test",
                model=feature)
    )

    pred_df = pd.concat([pred_train_df, pred_test_df]).reset_index(drop=True)
    return pred_df

def sample_squared_error(scores, y):
    """
    Calculate the squared error per sample depending on model scores
    """
    metadata_cols = [x for x in scores.columns if x.startswith("Metadata_")]
    scores_values = scores.drop(metadata_cols, axis="columns")
    
    all_squared_error = {}
    for cell_health_feature in scores_values.columns:
        y_subset_df = y.loc[:, cell_health_feature].dropna().T
        scores_subset = scores_values.loc[:, cell_health_feature].reindex(y_subset_df.index).T

        squared_error = (y_subset_df - scores_subset) ** 2
        all_squared_error[cell_health_feature] = squared_error
    
    return pd.DataFrame(all_squared_error).reindex(scores.index)

## 1) Load Models and Model Coefficients

For real data and shuffled model data.

In [4]:
consensus = "median"

In [5]:
model_dict, model_coef = load_models(consensus=consensus)
shuffle_model_dict, shuffle_model_coef = load_models(shuffle=True, consensus=consensus)

In [6]:
# Load Metadata Mapping File
data_dir = os.path.join("..", "1.generate-profiles", "data")
file = os.path.join(data_dir, "profile_id_metadata_mapping.tsv")
metadata_df = pd.read_csv(file, sep='\t')

metadata_df.head()

,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name
0,profile_0,A549,AKT1-1
1,profile_1,A549,AKT1-2
2,profile_2,A549,ARID1B-1
3,profile_3,A549,ARID1B-2
4,profile_4,A549,ATF4-1


## 2) Load Training and Testing Data

In [7]:
x_train_df, x_test_df, y_train_df, y_test_df = load_train_test(drop_metadata=True, consensus=consensus)

## 3) Output Model Coefficients

In [8]:
# Extract all model coefficients and output to file
coef_df = pd.DataFrame(model_coef)
coef_df.index = x_test_df.columns
coef_df.index.name = "features"

file = os.path.join("results",
                    "all_model_coefficients_{}.tsv".format(consensus))
coef_df.to_csv(file, sep='\t', index=True)

print(coef_df.shape)
coef_df.head(2)

(1598, 70)


,cell_health_median_target_vb_live_cell_roundness,cell_health_median_target_cc_g1_plus_g2,cell_health_median_target_cc_g2_ph3_neg_n_objects,cell_health_median_target_cc_g2_plus_all_m,cell_health_median_target_cc_cc_g2_ph3_neg_mean,cell_health_median_target_cc_edu_pos_alexa647_intensity_nucleus_area_sum,cell_health_median_target_cc_edu_pos_n_spots_per_nucleus_area_mean,cell_health_median_target_vb_percent_all_early_apoptosis,cell_health_median_target_cc_g2_ph3_pos_high_n_spots_h2ax_mean,cell_health_median_target_cc_cc_high_n_spots_h2ax_mean,...,cell_health_median_target_cc_all_n_spots_per_nucleus_area_mean,cell_health_median_target_vb_ros_mean,cell_health_median_target_cc_polyploid_n_spots_per_nucleus_area_mean,cell_health_median_target_cc_polyploid_n_spots_mean,cell_health_median_target_cc_all_n_spots_mean,cell_health_median_target_cc_edu_pos_n_objects,cell_health_median_target_cc_all_nucleus_roundness_mean,cell_health_median_target_cc_mitosis_ph3_neg_n_objects,cell_health_median_target_vb_num_live_cells,cell_health_median_target_cc_mitosis_ph3_neg_n_spots_mean
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,...,-0.0,-0.0,-0.0,0.0,-0.0,0.0,0.063351,0.0,-0.0,-0.0
Cells_AreaShape_Center_X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.000000,0.0,0.0,0.0


In [9]:
# Extract all model coefficients and output to file
shuffle_coef_df = pd.DataFrame(shuffle_model_coef)
shuffle_coef_df.index = x_test_df.columns
shuffle_coef_df.index.name = "features"

file = os.path.join("results",
                    "all_model_coefficients_shuffled_{}.tsv".format(consensus))
shuffle_coef_df.to_csv(file, sep='\t', index=True)

print(shuffle_coef_df.shape)
shuffle_coef_df.head(2)

(1598, 70)


,cell_health_median_target_vb_infection_percentage,cell_health_median_target_cc_cc_ph3_neg_hoechst_late_mitosis_mean,cell_health_median_target_cc_edu_pos_n_objects,cell_health_median_target_cc_mitosis_ph3_neg_n_objects,cell_health_median_target_cc_g2_ph3_pos_n_objects,cell_health_median_target_vb_ros_mean,cell_health_median_target_vb_live_cell_area,cell_health_median_target_cc_edu_pos_n_spots_mean,cell_health_median_target_cc_g1_n_spots_mean,cell_health_median_target_cc_all_high_n_spots_h2ax_mean,...,cell_health_median_target_cc_cc_g2_ph3_neg_mean,cell_health_median_target_vb_percent_dead,cell_health_median_target_cc_all_n_spots_mean,cell_health_median_target_cc_polynuclear_high_n_spots_h2ax_mean,cell_health_median_target_cc_g2_ph3_pos_n_spots_mean,cell_health_median_target_cc_polynuclear_n_spots_mean,cell_health_median_target_cc_g2_ph3_neg_n_objects,cell_health_median_target_cc_mitosis_ph3_pos_n_objects,cell_health_median_target_cc_g2_g1,cell_health_median_target_vb_percent_all_late_apoptosis
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,...,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0
Cells_AreaShape_Center_X,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,...,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0


## 4) Apply all models

For real and shuffled data.

In [10]:
all_scores = []
all_shuffle_scores = []
for cell_health_feature in model_dict.keys():
    # Apply Real Model Classifiers
    model_clf = model_dict[cell_health_feature]
    pred_df = apply_model(model=model_clf,
                          feature=cell_health_feature,
                          train_x=x_train_df,
                          test_x=x_test_df)
    all_scores.append(pred_df)
    
    # Apply Shuffled Model Classifiers
    shuffle_model_clf = shuffle_model_dict[cell_health_feature]
    shuffle_pred_df = apply_model(model=shuffle_model_clf,
                                  feature=cell_health_feature,
                                  train_x=x_train_df,
                                  test_x=x_test_df)
    all_shuffle_scores.append(shuffle_pred_df)

## 5) Concatenate scores with Metadata

In [11]:
# Concatenate real data scores
all_scores = (
    pd.concat(all_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_scores = (
    metadata_df.merge(all_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_scores.index = all_scores.Metadata_profile_id
all_scores = all_scores.drop("Metadata_profile_id", axis="columns")

# Remove prefix of variable columns
strip_text = "cell_health_{}_target_".format(consensus)
all_scores.columns = [x.replace(strip_text, "") for x in all_scores.columns]

# Output file
file = os.path.join("results",
                    "all_model_predictions_{}.tsv".format(consensus))
all_scores.to_csv(file, sep='\t', index=True)

print(all_scores.shape)
all_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.194252,0.770066,0.432754,0.144081,0.373440,0.390171,-0.008175,...,-0.117877,0.043554,0.100592,-0.018492,0.072148,0.017899,0.033002,-0.018988,0.272452,0.782514
profile_1,A549,AKT1-2,train,0.096767,0.310430,0.069082,0.267847,0.213499,0.153805,-0.186158,...,0.136455,0.083848,0.092258,0.162315,0.068527,0.126104,0.127216,-0.126984,0.249835,0.656351


In [12]:
# Concatenate shuffled data scores
all_shuffle_scores = (
    pd.concat(all_shuffle_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_shuffle_scores = (
    metadata_df.merge(all_shuffle_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_shuffle_scores.index = all_shuffle_scores.Metadata_profile_id
all_shuffle_scores = all_shuffle_scores.drop("Metadata_profile_id", axis="columns")

# Remove prefix of variable columns
strip_text = "cell_health_{}_target_".format(consensus)
all_shuffle_scores.columns = [x.replace(strip_text, "") for x in all_shuffle_scores.columns]

# Output file
file = os.path.join("results",
                    "all_model_predictions_shuffled_{}.tsv".format(consensus))
all_shuffle_scores.to_csv(file, sep='\t', index=True)

print(all_shuffle_scores.shape)
all_shuffle_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.232396,0.107642,0.076150,-0.199261,0.190197,0.260406,0.204475,...,-0.073629,0.111316,0.098742,0.084216,0.065425,0.074725,0.129232,-0.170322,-0.061968,0.023458
profile_1,A549,AKT1-2,train,0.283487,0.204544,0.085797,-0.164824,0.222316,0.260406,0.163884,...,-0.078440,0.130736,0.098742,0.016159,0.066085,0.069688,0.097255,-0.144882,-0.066892,0.038054


## 6) Calculate the Squared Error of Individual Samples

For real and shuffled data

In [13]:
y_df = pd.concat([y_train_df, y_test_df]).reindex(all_scores.index)

print(y_df.shape)
y_df.head(2)

(357, 70)


,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,cc_all_nucleus_roundness_mean,cc_cc_edu_pos_mean,cc_cc_g1_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,0.008156,0.587977,0.01882,0.381501,0.176564,0.187675,-0.170616,0.039147,0.085392,0.040949,...,0.399842,0.000000,0.000000,0.000000,-0.118976,-0.132871,-0.121090,0.132882,0.806970,1.293984
profile_1,0.056667,1.264627,0.24145,0.568443,0.235304,0.372684,-0.276888,-0.183445,0.883316,-1.024142,...,0.101670,0.318027,0.132751,0.467027,0.621374,0.100032,0.074036,-0.099917,0.558041,1.151867


In [14]:
all_score_error = sample_squared_error(scores=all_scores, y=y_df)

all_score_error = (
    metadata_df.merge(all_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results",
                    "all_model_sample_squared_error_{}.tsv".format(consensus))
all_score_error.to_csv(file, sep='\t', index=False)

print(all_score_error.shape)
all_score_error.head(2)

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.034631,0.033156,0.171341,0.056368,0.038760,0.041005,0.026387,...,0.268033,0.001897,0.010119,0.000342,0.036528,0.022732,0.023745,0.023064,0.285710,0.261601
1,profile_1,A549,AKT1-2,0.001608,0.910491,0.029711,0.090358,0.000475,0.047908,0.008232,...,0.001210,0.054839,0.001640,0.092850,0.305640,0.000680,0.002828,0.000733,0.094991,0.245537


In [15]:
all_shuffle_score_error = sample_squared_error(scores=all_shuffle_scores, y=y_df)

all_shuffle_score_error = (
    metadata_df.merge(all_shuffle_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results",
                    "all_model_sample_squared_error_shuffled_{}.tsv".format(consensus))
all_shuffle_score_error.to_csv(file, sep='\t', index=False)

print(all_shuffle_score_error.shape)
all_shuffle_score_error.head()

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.050284,0.230722,0.003287,0.337285,0.000186,0.005290,0.140693,...,0.224174,0.012391,0.009750,0.007092,0.034004,0.043096,0.062661,0.091932,0.755053,1.614235
1,profile_1,A549,AKT1-2,0.051447,1.123775,0.024228,0.537680,0.000169,0.012606,0.194280,...,0.032440,0.035078,0.001157,0.203282,0.308346,0.000921,0.000539,0.002022,0.390541,1.240580
2,profile_2,A549,ARID1B-1,0.006693,0.619379,0.000105,0.174884,0.003084,0.048971,0.259567,...,0.078047,0.038643,0.004411,0.004668,0.286468,0.005216,0.039325,0.000061,0.153808,0.001152
3,profile_3,A549,ARID1B-2,0.044707,0.011251,0.051064,0.093901,0.515359,0.301510,0.107236,...,0.272981,0.000593,0.001593,0.032285,0.095200,0.014476,0.055421,0.041921,0.038849,0.000061
4,profile_4,A549,ATF4-1,17.250360,0.004613,7.726477,4.274692,5.035212,6.856712,4.985655,...,5.506620,0.000620,0.009750,0.135282,0.026209,0.126680,0.143393,0.003830,0.582192,0.001354
